In [1]:
#Walkthrough: https://www.youtube.com/watch?v=54zkYWa9uu0

Search for truck 006565500C

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
driver = webdriver.Chrome()

What URL are we going to use? Not necessarily the launch page but the search result. Less clicking through.
Result is here: https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565500C
Search is here: https://www.tdlr.texas.gov/tools_search/mccs_search.asp
With search page on this occassion.


Why use Selenium? We don't have to. The site is weird if you use just requests. Scrape the page, displaying the 
The business name
Phone number
License status
Physical address
You should do. post requests by now

For physicla address: ask me on the board and I'll give you a secred trick about the situations like this.

In [33]:
driver.get('https://www.tdlr.texas.gov/tools_search')

In [6]:
#find_element(By.XPATH, "//*[@id="mcrbutton"]")

In [30]:
#//*[@id="mcrdata"]
#use single quotes
tdlr_input = driver.find_element(By.XPATH, '//*[@id="mcrdata"]')
#tdlr_input = driver.find_element_by_xpath('//*[@id="mcrdata"]')
tdlr_input
tdlr_input.send_keys("006565500C")

In [31]:
#//*[@id="submit3"]
search_button = driver.find_element(By.XPATH, '//*[@id="submit3"]')
search_button.click()

In [49]:
#//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]
business_name = driver.find_element(By.XPATH, "//*[contains(text(), 'Name')]/..")
#think Python: remove name and strip of spaces
business_name.text.replace('Name:', "").strip()

'A&NTOWINGLLC'

In [46]:
#//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]
phone_number = driver.find_element(By.XPATH, '//*[contains(text(), "Phone")]/..')
phone_number.text.replace('Phone:', "").strip()

'2106678546'

In [51]:
#//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font/font
#license_status = driver.find_element(By.XPATH, '//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]/font/font')
license_status = driver.find_element(By.XPATH, '//*[contains(text(), "Status")]/..')
license_status.text.replace('Status:', "").strip()

'Active'

In [55]:
#//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]
#physical_address = driver.find_element(By.XPATH, '//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]')
physical_address = driver.find_element(By.XPATH, '//*[contains(text(), "Physical")]/..')
physical_address.text

'Carrier Type:  Tow Truck Company\nNumber of Active Tow Trucks:   1\n\nAddress Information\nMailing:\n10838 DEEPWATER BAY\nSAN ANTONIO, TX. 78251\n\nPhysical:\n1341 GAYLE LN\nPOTEET, TX. 78065'

In [58]:
print(physical_address.text.split(":")[-1].strip())

1341 GAYLE LN
POTEET, TX. 78065


## Using .apply to find data about SEVERAL tow truck companies
The file trucks_subset.csv has information about the trucks, we'll use it to find pages to scrape
#### Open up trucks-subset.csv and save it into a dataframe

In [18]:
import pandas as pd
df = pd.read_csv('trucks-subset.csv')
df.head()

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


### Use .apply to go through each row of the dataset, printing out information about each tow truck company.
* the business name
* Phone number
* License status
* Physical address

Just it out for now.

* TIP: use .apply and a function
* TIP: if you need help with .apply, look at the "Using apply in pandas" notebook


In [34]:
df.head()

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


In [78]:
def process_truck(row):
    driver = webdriver.Chrome()
    
    # Visit the search page
    driver.get('https://www.tdlr.texas.gov/tools_search')
    
    # Fill out the form and submit it
    tdlr_input = driver.find_element(By.XPATH, '//*[@id="mcrdata"]')
    #tdlr_input = driver.find_element(By.LINK_TEXT, 'TDLR Number')
    tdlr_input.send_keys(row['TDLR Number'])
    
    driver.find_element(By.XPATH, '//*[@id="submit3"]').click()
        
    business_name = driver.find_element(By.XPATH, "//*[contains(text(), 'Name')]/..")
    #print("Name", business_name.text.replace('Name:', "").strip())
    phone_number = driver.find_element(By.XPATH, '//*[contains(text(), "Phone")]/..')
    #print("Phone", phone_number.text.replace('Phone:', "").strip())
    license_status = driver.find_element(By.XPATH, '//*[contains(text(), "Status")]/..')
    #print("Status", license_status.text)
    physical_address = driver.find_element(By.XPATH, '//*[contains(text(), "Physical")]/..')
    #print("Address", physical_address.text.split(":")[-1].strip())
    
    #print("---------")
    
    return pd.Series({
        'name': business_name.text.replace('Name:', "").strip(),
        'phone': phone_number.text.replace('Phone:', "").strip(),
        'status': license_status.text.replace('Status:', "").strip(),
        'address': physical_address.text.split(":")[-1].strip()
        
    })

complete_df = df.apply(process_truck, axis=1).join(df)

In [71]:
complete_df

,name,phone,status,address,TDLR Number
0,H & A TOWING LLC,512-999-8883,Active,"11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753",006565540C
1,24/7TOWINGANDRECOVERYLLC,4325576733,Active,"3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705",0654479VSF
2,A&NTOWINGLLC,2106678546,Active,"1341 GAYLE LN\nPOTEET, TX. 78065",006564940C


In [75]:
complete_df.to_csv('TXTowTrucks.csv', index=False)

In [76]:
!ls

HW - Mines.ipynb
HW - Tow Trucks.ipynb
HW - Updated Chicago Building Records Part 1.ipynb
HW - Updated Chicago Building Records Part 2.ipynb
Juhana - Tow Trucks.ipynb
Juhana Texas Tow Trucks Walkthrough.ipynb
mines-subset.csv
trucks-subset.csv
TXTowTrucks.csv


### Check for weird columns and rows

In [77]:
new_df = pd.read_csv("TXTowTrucks.csv")
new_df

,name,phone,status,address,TDLR Number
0,H & A TOWING LLC,512-999-8883,Active,"11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753",006565540C
1,24/7TOWINGANDRECOVERYLLC,4325576733,Active,"3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705",0654479VSF
2,A&NTOWINGLLC,2106678546,Active,"1341 GAYLE LN\nPOTEET, TX. 78065",006564940C


In [83]:
def process_truck(row):
    
    # Visit the search page
    driver.get('https://www.tdlr.texas.gov/tools_search')
    
    # Fill out the form and submit it
    tdlr_input = driver.find_element(By.XPATH, '//*[@id="mcrdata"]')
    #tdlr_input = driver.find_element(By.LINK_TEXT, 'TDLR Number')
    tdlr_input.send_keys(row['TDLR Number'])
    
    driver.find_element(By.XPATH, '//*[@id="submit3"]').click()

    
    business_name = driver.find_element(By.XPATH, "//*[contains(text(), 'Name')]/..")
    print("Name", business_name.text.replace('Name:', "").strip())
    phone_number = driver.find_element(By.XPATH, '//*[contains(text(), "Phone")]/..')
    print("Phone", phone_number.text.replace('Phone:', "").strip())
    license_status = driver.find_element(By.XPATH, '//*[contains(text(), "Status")]/..')
    print("Status", license_status.text.replace('Status:', "").strip())
    physical_address = driver.find_element(By.XPATH, '//*[contains(text(), "Physical")]/..')
    print("Address", physical_address.text.split(":")[-1].strip())
    
    print("---------")
driver = webdriver.Chrome()
df.apply(process_truck, axis=1)
driver.close()

Name H & A TOWING LLC
Phone 512-999-8883
Status Active
Address 11710 JOSEPH CLAYTON DR
AUSTIN, TX. 78753
---------
Name 24/7TOWINGANDRECOVERYLLC
Phone 4325576733
Status Active
Address 3601 N COUNTY ROAD 1148
MIDLAND, TX. 79705
---------
Name A&NTOWINGLLC
Phone 2106678546
Status Active
Address 1341 GAYLE LN
POTEET, TX. 78065
---------
